# **Import Statements**

In [307]:
import torch
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd



mat = scipy.io.loadmat('eyeliddata.mat', simplify_cells = True)
torch.cuda.is_available()

True

# **Data Reorganization** <br>
Reorganizing provided matlab file and interpolating over recoverable NaN labels

For a single trial, 

- Marker kinematics are provided at 400 Hz, shape (300, 3) --> 0.75 s
- EMG information is provided at 6103.5 Hz, shape (4564,) --> 0.7478 s

Marker and EMG data are time-synced. Blinking starts at around 0.25 seconds, which corresponds to point 100 for the kinematics data and point 1525 in the EMG data

Since only electrodes u1, u2, u3, u4, and t2 were used in common across all participants, our input to the model (without spectral preprocessing) will be of shape (4564, N, 5). Since only markerset x1, x2, A, B, C, D, E were used in common across all participants, our labels will be of shape (N, 7, 300, 3). In other words, $x \in R^{T_{\mathrm{in}}\times N \times C_{\mathrm{in}}}$, where $T_{\mathrm{in}}$ is the number of timesteps (4564, in this case), N is the batch size, and $C_{\mathrm{in}}$ is the number of channels (5, in this case), while $y \in R^{T_{\mathrm{out}}\times 3\times N \times C_{\mathrm{out}}}$, where where $T_{\mathrm{out}}$ is the number of timesteps (300 x 3, in this case), N is the batch size, and $C_{\mathrm{out}}$ is the number of channels (7, in this case).

To reorganize the data for train-test-val splits, the Nth data/label pair should contain the timeseries for emg electrodes u1, u2, u3, u4, t2/the timeseries for markerset x1, x2, A, B, C, D, E. In addition to this, a static identification label is generated which contains an identifying string combining the subject, blink type, and trial number (ie: 'sub1_spon#23'), a boolean hyperparameter indicating eye side (True = Right Eye, False = Left Eye); and a flag to identify whether the trial contains NaN values. 

In [353]:
'''
Reorganize Data
'''
eye_bool = np.asarray([True, True, False, True, True, False, False, False])
electrode_list = ['u1', 'u2', 'u3', 'u4', 't2']
marker_list = ['x1', 'x2', 'u1', 'u2', 'u3', 'u4', 'u5']


#Collect Data
data = np.zeros((4564, 1, 5))
for blink_key in mat['ForSamantha']['emg_with_notchfilter']:
    for sub_key in mat['ForSamantha']['emg_with_notchfilter'][blink_key]:
        trial = []
        for electrode_key in electrode_list:
            trial.append(mat['ForSamantha']['emg_with_notchfilter'][blink_key][sub_key][electrode_key])
        trial = np.dstack(trial) #reshape to (4564, T, 5) where T is number of trials
        data = np.hstack((data, trial)) #stack all trials along 1st axis --> (4564, N, 5)


#Collect Label/Identifier
identifier = []
label = np.zeros((300, 3, 1, 7))
for blink_key in mat['ForSamantha']['kinem']:
    for sub_key in mat['ForSamantha']['kinem'][blink_key]:
        trial = []
        for marker_key in marker_list:
            trial.append(mat['ForSamantha']['kinem'][blink_key][sub_key][marker_key])
        trial = np.stack(trial, axis = -1) #reshape to (300, 3, T, 7) where T is number of trials

        #Identifier markers for trials with incomplete data
        throwaway = False
        if trial.shape[0] != 300:
            throwaway = True

        #If not discarding a trial, stack it's label
        if not(throwaway):
            label = np.dstack((label, trial)) #stack all trials along 2nd axis --> (300, 3, N, 7)
        
        #Generate ID for each trial
        for num in range(trial.shape[2]):
            id_string = sub_key + '_' + blink_key + '#' + str(num + 1) 
            id_eye = eye_bool[int(sub_key.split('b')[1]) - 1]
            identifier.append((id_string, id_eye, False, throwaway))


#Remove placeholder
X = data[:, 1:, :] 
y = label[:, :, 1:, :]
identifier = np.asarray(identifier)


#Remove throwaway trials from data/identifier
idx = np.where(identifier[:,3] == 'True')
X = np.delete(X, idx, axis = 1)
identifier = np.delete(identifier, idx, axis = 0)
identifier = identifier[:, :3]


'''
Handling NaN Values by Trial
'''
def cubic_interp(trial):
    interp_trial = np.empty(np.shape(trial))
    for i in range(trial.shape[-2]):
        for j in range(trial.shape[-1]):
            df = pd.Series(trial[:, i, j])
            interp_trial[:, i, j] = np.asarray(df.interpolate(method = 'cubic', limit = 299))
    return interp_trial


#separate NaN indices 
timesteps, ccords, n_trial, n_channel = np.where(np.isnan(y))
removal = []
for trial in set(n_trial):
    if np.any(np.isnan(y[299, :, trial, :])):
        removal.append(trial)
    else:
        y[:, :, trial, :] = cubic_interp(y[:, :, trial, :])
        identifier[trial, 2] = True


#Remove un-interpolatable trials from data
X = np.delete(X, removal, axis = 1)
y = np.delete(y, removal, axis = 2)
identifier = np.delete(identifier, removal, axis = 0)


'''
Shape Validation
'''
#Validate shapes
print("Data Shape: ", X.shape)
print("Label Shape: ", y.shape)
print("Identifier Shape: ", identifier.shape)
assert X.shape[0] == 4564, "X Shape Invalid: Should have 4564 timesteps"
assert X.shape[2] == 5, "X Shape Invalid: Should have 5 Channels"
assert y.shape[0] == 300, "y Shape Invalid: Should have 300 timesteps"
assert y.shape[1] == 3, "y Shape Invalid: Each timestep should have 3 (cartesian) dimensions"
assert y.shape[3] == 7, "y Shape Invalid: Should have 7 Channels"
assert X.shape[1] == y.shape[2] == identifier.shape[0], "Shapes Invalid: The trial dimension of X, y, and identifier needs to match"
print("Shapes Validated!")

Data Shape:  (4564, 986, 5)
Label Shape:  (300, 3, 986, 7)
Identifier Shape:  (986, 3)
Shapes Validated!


# **Train-Val-Test Splits**

In [400]:
train = 0.7
val = 0.2
test = 0.1
np.random.seed(42) # for consistent train/val/test splits


#Calculate useful numbers
num_trials = identifier.shape[0]
num_train = int(train*num_trials)
num_val = int(val*num_trials)
num_test = int(test*num_trials)


#All possible trial indices
available_idx = np.arange(num_trials)


#Make sure interpolated labels do not affect ability to verify model performance
idx = np.where(identifier[:,2] == 'True')[0]
X_train = X[:, idx, :]
y_train = y[:, :, idx, :]
id_train = identifier[idx, :2]


#Remove indices we should no longer access, then form indices
available_idx = np.delete(available_idx, idx)
np.random.shuffle(available_idx)
train_idx = available_idx[:num_train-len(idx)]
val_idx = available_idx[num_train-len(idx):num_train - len(idx) + num_val]
test_idx = available_idx[num_train - len(idx) + num_val:]


#combine interpolated training data with the rest of the training data
X_train = np.hstack((X_train, X[:, train_idx, :]))
y_train = np.dstack((y_train, y[:, :, train_idx, :]))
id_train = np.vstack((id_train, identifier[train_idx, :2]))


#Create validation data
X_val = X[:, val_idx, :]
y_val = y[:, :, val_idx, :]
id_val = identifier[val_idx, :2]


#Create test data
X_test = X[:, test_idx, :]
y_test = y[:, :, test_idx, :]
id_test = identifier[test_idx, :2]